## Deep Learning Course (980)
## Assignment Two: Convolutional Neural Networks

__Assignment Goals__:
- Design and implementation of CNNs.
- Understanding the different effects of linear and nonlinear activation functions.
- CNN visualization.

In this assignment, you will be asked to learn a CNN model for an image dataset. Different experiments will help you achieve a better understanding of CNNs. 

__DataSet__: the dataset consists of around 9K images (some grayscale and some RGB) belonging to 101 classes. The shape of each image is (64,64,3). Every image is labeled with one of the classes. The image file is contained in the folder named after the class name.

1. We are aiming to lean a CNN on the dataset. Download the dataset, and use TensorFlow to implement LeNet5 to classify instances. Use a one-hot encoding for labels. Split the dataset into training (90 percent) and validation (10 percent) and report the model loss (cross-entropy) and accuracy on both sets. Try to improve model accuracy on the validation dataset by tuning the model hyperparameters. You can use regularization to prevent overfitting. The model performance is a part of the overall evaluation (35 points). The LeNet5 configuration is:
  - Convolutional layer (kernel size 5 x 5, 32 filters, stride 1 x 1 and followed by ReLU)
  - Max Pooling layer subsampling by 4 and stride 4 x 4 
  - Convolutional layer (kernel size 5 x 5, 64 filters, stride 1 x 1 and followed by ReLU)
  - Max Pooling layer subsampling by 4 and stride 4 x 4
  - Fully Connected ReLU layer that has input 7764 and output 1024 
  - Fully Connected ReLU layer that has input 1024 and output 84 
  - Fully Connected Softmax layer that has input 84 and output which is equal to the number of classes (one node for each of the classes).
2. What happens if we use a linear activation function in all convolutions and dense layers (except softmax in the last fully connected layer)? Compare training and validation loss (cross-entropy) after and before changing the activation function. Can we compensate for the effect of removing the non-linear activation function by adding more convolution linear layers? Explain your answer. (20 points)
3. There are several approaches to understand and visualize convolutional Networks, including visualizing the activations and layers weights. The most straight-forward visualization technique is to show the activations of the network during the forward pass. The second most common strategy is to visualize the weights. The weights are useful to visualize because well-trained networks usually display nice and smooth filters without any noisy patterns.  Please visualize the filters (i.e., the first layer convolution weights) which your CNN has learned for this task. Use the trained non-linear CNN model which you implemented in section 1. (Reference: for more information we recommend the course notes on "Visualizing what ConvNets learn" http://cs231n.github.io/understanding-cnn/. More advanced techniques can be found in "Visualizing and Understanding Convolutional Networks" paper by Matthew D.Zeiler and Rob Fergus.) (35 points)

__NOTE__: Please use Jupyter Notebook. The notebook should include the final code, results and your answers. You should submit your Notebook in (.pdf or .html) and .ipynb format. (10 points) 

__Instructions__:

The university policy on academic dishonesty and plagiarism (cheating) will be taken very seriously in this course. Everything submitted should be your own writing or coding. You must not let other students copy your work. Spelling and grammar count.

Your assignments will be marked based on correctness, originality (the implementations and ideas are from yourself), clarification and test performance.


In [1]:
import os
import glob
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import keras
from keras import layers
from keras import regularizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# Load data
img_dir = "101_ObjectCategories_resized/"
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2,
                                   rotation_range=20, horizontal_flip=True, validation_split=0.10)
train_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=(64, 64),
    batch_size=128,
    class_mode='categorical',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=(64, 64),
    batch_size=128,
    class_mode='categorical',
    subset='validation')

Found 7857 images belonging to 101 classes.
Found 820 images belonging to 101 classes.


In [3]:
# Build model
weight_decay = 1e-4
model = keras.Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(64, 64, 3), kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(layers.MaxPooling2D(pool_size=(4), strides=(2)))
model.add(layers.MaxPooling2D(pool_size=(4), strides=(4)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
# model.add(layers.MaxPooling2D(pool_size=(4), strides=(2)))
model.add(layers.MaxPooling2D(pool_size=(4), strides=(4)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(units=1024, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Dense(units=84, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(layers.Dense(units=101, activation = 'softmax'))
model.summary()

# Linear activation
# model = keras.Sequential()
# model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), activation='linear', input_shape=(64, 64, 3)))
# model.add(layers.MaxPooling2D(pool_size=(4), strides=(4)))
# model.add(layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), activation='linear'))
# model.add(layers.MaxPooling2D(pool_size=(4), strides=(4)))
# model.add(layers.Flatten())
# model.add(layers.Dense(units=1024, activation='linear'))
# model.add(layers.Dense(units=84, activation='linear'))
# model.add(layers.Dense(units=101, activation = 'softmax'))
# model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)              

In [ ]:
# Training
result_root_folder = 'checkpoints'
checkpointer = keras.callbacks.ModelCheckpoint(
        filepath=result_root_folder+'/epoch-{epoch:02d}-accu-{val_accuracy:.4f}.hdf5',
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True, mode='auto', period=5)
logfile = keras.callbacks.CSVLogger(result_root_folder+'/train.log', append=False, separator=',')
adam = keras.optimizers.Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

history = model.fit_generator(
        train_generator,
        epochs=10,
        callbacks=[checkpointer, logfile],
        shuffle=True, 
        validation_data=validation_generator,
        workers=6
        )

1.I used several strategies to improve model accuracy:     
  * I applied data augmentation to increase the diversity of data available.     
  * I added dropout layers after each max pooling layer to reduce interdependent learning amongst the neurons and prevent overfitting     
  * I used regularization to prevent overfitting.    
  * I tried with different learning rates and picked 0.0005    
  * I compared performance of different MaxPooling2D strides(trained both on tride 4 and tride 2)    
 
The training logs are added to the end of this file. Because I my network training results are saved to local folder, so I appended the training log to the end. I trained both non-linear network and linear network for 200 epoches.

Accuracy and loss on train dataset and validation dataset:    
The best model I chose is on epoch 194, with train loss: 0.3915 - train accuracy: 0.9730 - validation loss: 1.8612 - validation accuracy: 0.6707

2.Compare training and validation loss with non-linear activation function and linear activation function.    
With linear activation function: on epoch 194, train loss: 0.6286 - train accuracy: 0.8261 - validation loss: 1.8855 - validation accuracy: 0.6159    

Linear model has a bigger training and validation loss. Validation accuracy of linear model is lower than that of non-linear model.    
We can't compensate for the effect of removing the non-linear activation function by adding more convolution linear layers.     
Because even though we add more convolution linear layers, it's still equivalent to a linear neural network without hidden layers, i.e., to a linear classifier. They have limited complexity and have less power to learn complex functional mappings from data. Non-linear functions have degree more than one. It helps us to learn complicated, high dimensional, non-linear data such as images and videos, etc. That's the reason we need non-linear activation functions. Also nonlinear activation functions are needed for the universal approximation theorem for neural networks to be valid. 

In [ ]:
# Displaying curves of loss and accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig(result_root_folder+'/accuracy.png')
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig(result_root_folder+'/loss.png')

Train and validation accuracy:    
![Visualize accuracy](accuracy.png "train and validation accuracy")

Train and validation loss:     
![Visualize loss](loss.png "train and validation loss")

In [ ]:
# Predict image
model.load_weights('epoch-150-accu-0.6756.hdf5')
test_dir = "test_set/"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(64, 64),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)
filenames = test_generator.filenames
out = model.predict_generator(test_generator)
preds_cls_idx = np.argmax(out,axis=1)
idx_to_cls = {v: k for k, v in train_generator.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(preds_cls_idx)
filenames_to_cls = list(zip(test_generator.filenames, preds_cls))
print(filenames_to_cls)

Found 20 images belonging to 6 classes.   
[ ('airplanes/download (2).jpeg', 'helicopter')
, ('airplanes/download (3).jpeg', 'helicopter')
, ('airplanes/download.jpeg', 'airplanes')
, ('butterfly/image_0008.jpg', 'butterfly')
, ('butterfly/image_0006.jpg', 'airplanes')
, ('dolphin/images (1).jpeg', 'dolphin')
, ('dolphin/images (3).jpeg', 'hawksbill')
, ('dolphin/images.jpeg', 'okapi')
, ('elephant/images (1).jpeg', 'bonsai')
, ('elephant/images (2).jpeg', 'cannon')
, ('elephant/images (3).jpeg', 'elephant')
, ('elephant/images (4).jpeg', 'elephant')
, ('saxophone/images (1).jpeg', 'saxophone')
, ('saxophone/images (5).jpeg', 'saxophone')
, ('saxophone/images.jpeg', 'saxophone')
, ('sunflower/download.jpeg', 'sunflower')
, ('sunflower/images (1).jpeg', 'sunflower')
, ('sunflower/images (2).jpeg', 'sunflower')
, ('sunflower/images.jpeg', 'sunflower')
, ('sunflower/output_y8xuN_HBzGup_0.png', 'sunflower')]    

I tested the trained network on 20 unseen images of 6 classes(images I downloaded from internet). 13 out of 20 images are correctly predicted. The test accuracy of my net is 0.65.

In [ ]:
# Visualize filters for the first layer
model.load_weights('epoch-150-accu-0.6756.hdf5')
filters, biases = model.layers[0].get_weights()
# normalize filter values to 0-1
filters = (filters - filters.min()) / (filters.max() - filters.min()) # (5, 5, 3, 32)
n = 6
idx = 1
while idx <= filters.shape[3]:
    f = filters[:, :, :, idx-1]
    ax = plt.subplot(n, n, idx)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(f)
    idx += 1
plt.show()

# Visualize feature map for the first layer
test_img_path = './101_ObjectCategories_resized/butterfly/image_0008.jpg'
test_img = image.load_img(test_img_path, target_size=(64, 64))
test_img_tensor = image.img_to_array(test_img)
test_img_tensor = np.expand_dims(test_img_tensor, axis=0)
from keras.models import Model
new_model = Model(inputs=model.inputs, outputs=model.layers[0].output)
feature_map = new_model.predict(test_img_tensor) # (1, 60, 60, 32)
# print(feature_map.shape)
n = 6
idx = 1
while idx <= feature_map.shape[3]:
    ax = plt.subplot(n, n, idx)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(feature_map[0, :, :, idx-1], cmap='gray')
    idx += 1
plt.show()

I visualized filters and activations for the first layer.

Filters:    
![Visualize filters](Figure_1.png "filters")

Activations:     
![Visualize activations](Figure_2.png "activations")

Non-linear model training log:

epoch,accuracy,loss,val_accuracy,val_loss
0,0.1631666,4.311724271151618,0.1902438998222351,3.980665922164917
1,0.2830597,3.6740360523705506,0.26951220631599426,3.2702343463897705
2,0.33180603,3.3008742729974516,0.3573170602321625,3.4276018142700195
3,0.38691613,2.98198370868595,0.38780486583709717,2.801422119140625
4,0.43489882,2.6846850867548775,0.4390243887901306,2.6685352325439453
5,0.46951762,2.4752478981891994,0.4560975730419159,2.7007627487182617
6,0.50146365,2.307494669481287,0.48902440071105957,3.0659165382385254
7,0.52691865,2.155354170713076,0.5207316875457764,2.3149282932281494
8,0.55250096,2.031689967911283,0.5341463685035706,2.0586953163146973
9,0.5696831,1.9384845112728621,0.5597561001777649,2.321289539337158
10,0.58571976,1.8580498000175667,0.5658536553382874,1.9564235210418701
11,0.6012473,1.7641867628014996,0.5817072987556458,1.5209027528762817
12,0.6174112,1.6969604343194853,0.5585365891456604,1.681725263595581
13,0.6347206,1.6025700280877855,0.5878048539161682,1.9811944961547852
14,0.6568665,1.5467075920141393,0.6146341562271118,2.197054147720337
15,0.6670485,1.4808307868220263,0.5939024686813354,1.997930645942688
16,0.67213947,1.4464390521834214,0.6158536672592163,1.9714241027832031
17,0.69288534,1.3675037046205274,0.6134146451950073,1.5601552724838257
18,0.6992491,1.3380309729626483,0.6317073106765747,1.454251766204834
19,0.7150312,1.2705482166202162,0.6365853548049927,2.529308557510376
20,0.7238132,1.2251958808508085,0.6378048658370972,2.4531564712524414
21,0.7266132,1.222067337265539,0.6146341562271118,1.5315194129943848
22,0.73883164,1.1676108961010063,0.6170731782913208,1.8461209535598755
23,0.7567774,1.1102438569903388,0.6231707334518433,1.9340882301330566
24,0.761232,1.0822699715561033,0.6121951341629028,1.695990800857544
25,0.76581395,1.0691036302745685,0.6231707334518433,2.008140802383423
26,0.78566885,1.0023671872776339,0.6451219320297241,2.457620620727539
27,0.7840142,0.9978711776823219,0.6402438879013062,1.4619754552841187
28,0.7959781,0.9564716277632896,0.6426829099655151,0.9290918111801147
29,0.79775995,0.929142948824652,0.6560975313186646,1.4955352544784546
30,0.80603284,0.8859381474382901,0.6329268217086792,1.773698091506958
31,0.819524,0.8735125415331537,0.6439024209976196,1.4233126640319824
32,0.8248695,0.8504190506849547,0.6524389982223511,1.5194981098175049
33,0.8317424,0.8272046535937889,0.6524389982223511,1.7201740741729736
34,0.8349243,0.8249189185628882,0.6317073106765747,1.202378273010254
35,0.8391243,0.7841702132442305,0.6634146571159363,1.511149525642395
36,0.84866995,0.7645708471821119,0.6646341681480408,2.41660475730896
37,0.8524882,0.7566561206115086,0.6524389982223511,1.709114909172058
38,0.85503376,0.7497317869805555,0.6731707453727722,1.4427688121795654
39,0.87005216,0.7032890804429629,0.6414633989334106,2.1583704948425293
40,0.8701795,0.7176599350384633,0.6573171019554138,1.8891774415969849
41,0.87336135,0.679152689605811,0.6634146571159363,2.0583713054656982
42,0.8844343,0.6616708841945312,0.6646341681480408,2.4312944412231445
43,0.8883798,0.6434678124671402,0.6670731902122498,1.5342811346054077
44,0.879216,0.6560853765531853,0.6609756350517273,1.9695335626602173
45,0.8836706,0.6444665135144125,0.6451219320297241,2.2342476844787598
46,0.88188875,0.6562276500106177,0.6475609540939331,2.137092351913452
47,0.8977981,0.6151853196566625,0.6463414430618286,2.120837688446045
48,0.887107,0.6452540406838497,0.6414633989334106,1.5749038457870483
49,0.904671,0.5969385189364942,0.6487804651260376,2.003281354904175
50,0.8980527,0.6296653699765636,0.6743902564048767,1.9711936712265015
51,0.9027619,0.6102407053558672,0.6573171019554138,2.208346128463745
52,0.90708923,0.5859733416376579,0.6548780202865601,2.0000388622283936
53,0.91307116,0.5689309634855265,0.6573171019554138,2.5578346252441406
54,0.9138348,0.5683199081679435,0.6560975313186646,3.054086208343506
55,0.91956216,0.5515438140963452,0.6597561240196228,2.2503817081451416
56,0.9116711,0.5760608275943818,0.6682927012443542,1.807830572128296
57,0.9219804,0.5490600207660039,0.6634146571159363,2.913220167160034
58,0.92999876,0.5245288686965036,0.6707317233085632,2.685682535171509
59,0.93050784,0.5255666628955991,0.6841463446617126,2.6005823612213135
60,0.9210895,0.5396709823040952,0.6609756350517273,2.718243360519409
61,0.92630774,0.5327347411300081,0.6499999761581421,2.4702401161193848
62,0.93292606,0.508256091546721,0.6658536791801453,2.0201961994171143
63,0.9279623,0.5272170850167646,0.6707317233085632,1.1990365982055664
64,0.9373807,0.49386456447186566,0.6512194871902466,2.061908721923828
65,0.9377625,0.4940975390804228,0.6621951460838318,1.6669553518295288
66,0.9317806,0.5256611665985095,0.6414633989334106,2.3682072162628174
67,0.9368716,0.5028686029576945,0.6512194871902466,2.0181524753570557
68,0.9330533,0.5049270342760934,0.6475609540939331,2.1946022510528564
69,0.9381443,0.49959893441009084,0.6707317233085632,1.9547057151794434
70,0.94272625,0.4793975307230763,0.6536585092544556,2.375697374343872
71,0.9422171,0.4915275405013745,0.6609756350517273,3.178051233291626
72,0.93699884,0.5014360277187252,0.6621951460838318,2.380044937133789
73,0.9406898,0.4897210603757443,0.6475609540939331,1.82628333568573
74,0.9437444,0.4839914793790381,0.6487804651260376,2.015233039855957
75,0.93648976,0.4951044387780666,0.6390243768692017,2.537898063659668
76,0.9457808,0.47203055311284087,0.6365853548049927,2.0083813667297363
77,0.94654447,0.47116321354508933,0.6597561240196228,1.8837835788726807
78,0.9464172,0.47531180330112993,0.6524389982223511,2.5979111194610596
79,0.94616264,0.47578263961739314,0.6621951460838318,2.785390615463257
80,0.9419626,0.4874886735350148,0.6524389982223511,2.5962014198303223
81,0.9479445,0.47088856390676564,0.6731707453727722,3.5473432540893555
82,0.9455263,0.47830821478717334,0.6451219320297241,2.190540075302124
83,0.94909,0.46940042989026715,0.6695122122764587,1.8833715915679932
84,0.9493445,0.4615522102156835,0.6695122122764587,2.5862228870391846
85,0.9495991,0.46371168888933684,0.6682927012443542,2.484375238418579
86,0.94921726,0.4696628558388402,0.6536585092544556,2.0670483112335205
87,0.94832635,0.47436557236223925,0.6560975313186646,2.0818419456481934
88,0.9520173,0.4598154089553098,0.6573171019554138,3.2406318187713623
89,0.9434899,0.4765536263044575,0.6487804651260376,2.1893160343170166
90,0.95278096,0.4512274381433778,0.6414633989334106,3.1121089458465576
91,0.9614357,0.4289130719014217,0.6621951460838318,1.852903962135315
92,0.9437444,0.4734553692525043,0.6304877996444702,2.2812767028808594
93,0.9509991,0.4575849288942493,0.6682927012443542,2.180528163909912
94,0.95545375,0.44641134368564633,0.6426829099655151,2.1945955753326416
95,0.95239913,0.4623467057196105,0.6682927012443542,1.2221640348434448
96,0.95698106,0.4512733013496093,0.6487804651260376,3.1918492317199707
97,0.95303553,0.4535429150234034,0.6499999761581421,1.5612016916275024
98,0.95074457,0.4623579127773068,0.6634146571159363,2.4795827865600586
99,0.9587629,0.43277873878140966,0.6499999761581421,2.4166300296783447
100,0.9536719,0.46005029868833797,0.6658536791801453,1.7173742055892944
101,0.95952654,0.43283831371489684,0.6414633989334106,2.6294312477111816
102,0.96372664,0.4262469919549914,0.6634146571159363,2.3900651931762695
103,0.9543083,0.4540420194322771,0.6390243768692017,1.5791922807693481
104,0.9526537,0.46156521170166925,0.6536585092544556,3.623826742172241
105,0.9582538,0.43412621758344355,0.6499999761581421,2.6286637783050537
106,0.9587629,0.44193882392656564,0.6402438879013062,2.3415513038635254
107,0.9596538,0.4288666287807619,0.6646341681480408,2.2784247398376465
108,0.9614357,0.42949499011768005,0.6695122122764587,1.7811181545257568
109,0.9587629,0.4330779578571361,0.6475609540939331,2.5879387855529785
110,0.9623266,0.42519904717668344,0.6682927012443542,2.2804672718048096
111,0.95532644,0.45258859083278347,0.6475609540939331,2.1760690212249756
112,0.95659924,0.43651129899550534,0.6682927012443542,2.593848943710327
113,0.960672,0.43326117131900316,0.6487804651260376,1.794370174407959
114,0.96398115,0.41699595201466094,0.6499999761581421,2.9340271949768066
115,0.9591447,0.4278644797788923,0.6548780202865601,2.3077120780944824
116,0.9571083,0.4392876001931912,0.6634146571159363,2.071988582611084
117,0.95774466,0.43660112976861,0.6573171019554138,2.9689948558807373
118,0.96410847,0.41317796190154904,0.6707317233085632,2.286869764328003
119,0.95990837,0.4292600725699096,0.6378048658370972,1.099389672279358
120,0.9551992,0.44818480331979366,0.6634146571159363,2.362968683242798
121,0.9632175,0.4241800612243848,0.6646341681480408,2.3624813556671143
122,0.95685375,0.43222280516717915,0.6560975313186646,2.6383755207061768
123,0.96372664,0.4188336896330433,0.6585366129875183,1.813072681427002
124,0.9633448,0.4174340027456173,0.6634146571159363,2.8714330196380615
125,0.9602902,0.43274723619802985,0.6634146571159363,1.9846327304840088
126,0.9607993,0.42836551701838965,0.6573171019554138,2.4830617904663086
127,0.96283567,0.4227963380038594,0.6573171019554138,2.0612857341766357
128,0.962963,0.42668830211250264,0.6560975313186646,3.185835123062134
129,0.96130836,0.4170430554595023,0.6597561240196228,2.1287927627563477
130,0.95698106,0.4408012524169411,0.6573171019554138,2.952265739440918
131,0.96245384,0.426250958269723,0.6378048658370972,2.6111249923706055
132,0.96665394,0.409338522912498,0.6609756350517273,2.1947288513183594
133,0.9623266,0.4187677077692065,0.6304877996444702,2.314669132232666
134,0.9670358,0.4130258017009916,0.6695122122764587,2.5294458866119385
135,0.9653812,0.41472401836074096,0.6609756350517273,1.5656168460845947
136,0.96398115,0.41557981690802737,0.6597561240196228,2.829411029815674
137,0.96754485,0.41317434781332335,0.6524389982223511,2.918755531311035
138,0.9633448,0.41609158204289753,0.6560975313186646,2.4209046363830566
139,0.97378135,0.38335179474664216,0.6682927012443542,3.3227274417877197
140,0.96601754,0.4022419537291115,0.6646341681480408,2.559128522872925
141,0.9632175,0.4164606328416875,0.6597561240196228,3.3999550342559814
142,0.96016294,0.427730349880966,0.6463414430618286,2.402829885482788
143,0.96487206,0.4167816300223434,0.6585366129875183,2.2841949462890625
144,0.96449023,0.4164191306858735,0.6609756350517273,2.8617637157440186
145,0.962963,0.42159845587162476,0.6475609540939331,3.2819859981536865
146,0.9576174,0.43219802344939295,0.6841463446617126,2.3589766025543213
147,0.9616902,0.4297110344164393,0.6695122122764587,1.8657530546188354
148,0.96843576,0.4078346398839578,0.6499999761581421,2.250765323638916
149,0.96665394,0.40859782991252114,0.6756097674369812,3.2156262397766113
150,0.9674176,0.40533369907793293,0.6597561240196228,2.2656497955322266
151,0.9653812,0.4138664665515905,0.6670731902122498,2.009681224822998
152,0.9597811,0.41869430769122523,0.6573171019554138,2.108060359954834
153,0.96449023,0.4145387922578054,0.6634146571159363,3.218644380569458
154,0.96805394,0.40870644778214266,0.6573171019554138,2.1980953216552734
155,0.96347207,0.4167466071804501,0.6621951460838318,3.0310122966766357
156,0.96614486,0.41032077551297713,0.6646341681480408,2.806198835372925
157,0.96729034,0.40492796626302807,0.6451219320297241,2.219632625579834
158,0.9642357,0.41588092297486284,0.6487804651260376,1.4073634147644043
159,0.96245384,0.41895345021435154,0.6609756350517273,2.949920654296875
160,0.96041745,0.4248049008841731,0.6682927012443542,2.5457546710968018
161,0.96869034,0.39917259756748313,0.6573171019554138,4.627795219421387
162,0.9635993,0.4240324030255367,0.6499999761581421,3.3681626319885254
163,0.96907216,0.40621619221438005,0.6573171019554138,2.633066415786743
164,0.9689449,0.4082903325125296,0.6524389982223511,1.7762466669082642
165,0.96665394,0.40473257579678446,0.6634146571159363,2.409268617630005
166,0.9685631,0.4065401035516153,0.6329268217086792,3.23585844039917
167,0.97110856,0.39353731934982084,0.6487804651260376,2.2841618061065674
168,0.9663994,0.40245636702494203,0.6524389982223511,3.8405869007110596
169,0.9616902,0.418872765742505,0.6365853548049927,2.5489189624786377
170,0.9656357,0.41124231450139254,0.6536585092544556,3.334686279296875
171,0.9713631,0.39393046021507006,0.6646341681480408,3.095360279083252
172,0.9688176,0.39849061702398364,0.6573171019554138,3.3130087852478027
173,0.9730177,0.3938429365938548,0.6512194871902466,3.674544334411621
174,0.9688176,0.3943820756075263,0.6658536791801453,2.923365592956543
175,0.9674176,0.4053953778801399,0.6609756350517273,2.8798694610595703
176,0.9760723,0.37960813580175934,0.6597561240196228,1.6876156330108643
177,0.9718722,0.39035974212149194,0.6426829099655151,3.0217950344085693
178,0.96372664,0.412450368875187,0.6451219320297241,3.1738924980163574
179,0.9662721,0.41548999909971057,0.6524389982223511,3.233506679534912
180,0.9683085,0.39880136537296984,0.6634146571159363,4.071393013000488
181,0.96869034,0.3968520045955627,0.6451219320297241,3.268439769744873
182,0.96970856,0.39738909968274444,0.6402438879013062,3.040571928024292
183,0.9718722,0.39232477376187014,0.6439024209976196,1.890153408050537
184,0.9750541,0.3770973732586169,0.6536585092544556,3.613063335418701
185,0.9712358,0.3860150632875573,0.6634146571159363,3.4626471996307373
186,0.97225404,0.39207542742938806,0.6560975313186646,3.372917890548706
187,0.9674176,0.4000539190482134,0.6682927012443542,3.732250928878784
188,0.96805394,0.4034976710249969,0.6463414430618286,2.409139394760132
189,0.9703449,0.3998773344808428,0.6536585092544556,2.9633755683898926
190,0.9642357,0.4136422402465466,0.6512194871902466,2.1309783458709717
191,0.9632175,0.42272291467422535,0.6695122122764587,3.517726421356201
192,0.96601754,0.4077727592437492,0.6499999761581421,2.3991775512695312
193,0.9718722,0.3880067919332957,0.6573171019554138,1.8939074277877808
194,0.9730177,0.39153388059137734,0.6707317233085632,1.861183853149414
195,0.96970856,0.3970657215707703,0.6682927012443542,2.5135600566864014
196,0.96309024,0.4126184024822322,0.6548780202865601,2.1694443225860596
197,0.97110856,0.39142140099394485,0.6658536791801453,2.647382974624634
198,0.96958125,0.3889054969565717,0.6621951460838318,2.1779026985168457
199,0.9689449,0.4040353267766659,0.6439024209976196,2.382054328918457


Linear model training log:
    
epoch,accuracy,loss,val_accuracy,val_loss
0,0.2131857,3.9146935772285882,0.23292683064937592,3.9071638584136963
1,0.3156421,3.2433879962324927,0.3280487656593323,3.3165459632873535
2,0.36451572,2.9227680710871073,0.36219513416290283,2.759111166000366
3,0.40002546,2.6863804544764682,0.37804877758026123,3.0531554222106934
4,0.42764413,2.532026568851323,0.4158536493778229,2.3191781044006348
5,0.4513173,2.3980714736343476,0.4268292784690857,3.061441659927368
6,0.47295406,2.3107765698017197,0.46219512820243835,2.6184141635894775
7,0.48809978,2.205038123812341,0.4719512164592743,2.5508487224578857
8,0.49586356,2.155790625068239,0.4878048896789551,2.6545615196228027
9,0.51597303,2.063124014097148,0.48170730471611023,2.437366008758545
10,0.526155,2.009945806783924,0.5097560882568359,2.029207944869995
11,0.53608245,1.9418833508842908,0.5060975551605225,2.149625301361084
12,0.54384625,1.9280619079319896,0.507317066192627,2.4157636165618896
13,0.5568283,1.8718320852141048,0.5304877758026123,2.111806631088257
14,0.56281024,1.812982155846718,0.5329268574714661,1.9132651090621948
15,0.5721013,1.762369738259934,0.5329268574714661,1.9702644348144531
16,0.5796105,1.732360790351488,0.5414634346961975,2.2670092582702637
17,0.5843197,1.7106876458711444,0.5585365891456604,1.8872835636138916
18,0.59361076,1.670176697214308,0.5524390339851379,2.1503825187683105
19,0.59972,1.6455357011468776,0.5658536553382874,1.5017346143722534
20,0.59984726,1.6256031030550764,0.5658536553382874,2.5176444053649902
21,0.6151203,1.574979635888389,0.5695121884346008,1.9407944679260254
22,0.6211022,1.5373025263662146,0.550000011920929,2.00323224067688
23,0.6323024,1.4995659020045065,0.5768292546272278,1.8438996076583862
24,0.6333206,1.4987115478552038,0.5731707215309143,2.0998647212982178
25,0.63217515,1.4953443308254075,0.5658536553382874,2.0512514114379883
26,0.64019346,1.4431608550433548,0.550000011920929,2.024047374725342
27,0.6363752,1.453540357921936,0.5817072987556458,1.7770875692367554
28,0.6471936,1.3977312307799608,0.5902438759803772,1.9181232452392578
29,0.6485936,1.4157796064466408,0.5902438759803772,1.765807032585144
30,0.658012,1.3697667715126767,0.5768292546272278,1.794662356376648
31,0.6613211,1.3531014099973173,0.6097561120986938,2.08858060836792
32,0.6586483,1.3380090006438317,0.5841463208198547,2.558952569961548
33,0.6653939,1.3173530427526847,0.5914633870124817,1.8862881660461426
34,0.6725213,1.3038169693306565,0.5926828980445862,1.6431344747543335
35,0.6802851,1.2817642434917516,0.5914633870124817,1.9130346775054932
36,0.6751941,1.293161707296465,0.5902438759803772,1.300600290298462
37,0.6871579,1.2631248893728082,0.6146341562271118,1.957468032836914
38,0.6767214,1.2781000393590385,0.5817072987556458,1.236966609954834
39,0.6867761,1.2359675648893065,0.5829268097877502,2.248145341873169
40,0.6853761,1.233216631013473,0.6036585569381714,2.0928876399993896
41,0.68575794,1.2239657464002522,0.5914633870124817,2.3835790157318115
42,0.6918671,1.1975620815404062,0.6170731782913208,1.6395056247711182
43,0.6889398,1.19280157953148,0.6268292665481567,1.5726649761199951
44,0.6956854,1.1623078917401035,0.6182926893234253,1.6518890857696533
45,0.6951763,1.1794803320367753,0.6195122003555298,1.9872525930404663
46,0.7043401,1.139444582332132,0.6097561120986938,1.3157564401626587
47,0.7037037,1.142419742577492,0.5939024686813354,2.1328158378601074
48,0.70383096,1.1429738733427692,0.6195122003555298,1.6535319089889526
49,0.7067583,1.1206050708546687,0.6000000238418579,1.7454966306686401
50,0.7152857,1.0879212066055148,0.6243902444839478,1.9471079111099243
51,0.70917654,1.1073151970313253,0.6000000238418579,1.3295376300811768
52,0.70955837,1.110972468416336,0.6036585569381714,1.4274719953536987
53,0.7185949,1.0711260907474298,0.6146341562271118,1.8154524564743042
54,0.7211404,1.087338504012007,0.6097561120986938,1.8130608797073364
55,0.710322,1.0951516241445878,0.6182926893234253,2.658182382583618
56,0.72521317,1.079705216691317,0.6256097555160522,1.2799897193908691
57,0.72368586,1.0698499410862805,0.6158536672592163,2.651984691619873
58,0.72037673,1.0444804735427689,0.6073170900344849,2.315058469772339
59,0.72126764,1.0599377956480809,0.6304877996444702,2.04447603225708
60,0.7273769,1.0378399716771498,0.6207317113876343,1.9455996751785278
61,0.7280132,1.023026197227006,0.6146341562271118,1.4192802906036377
62,0.73412246,1.0306559901846204,0.6060975790023804,1.9331635236740112
63,0.72750413,1.015801891200913,0.6024390459060669,1.7324821949005127
64,0.7286496,1.0233552073103274,0.6085366010665894,1.7419450283050537
65,0.737177,0.9940068165628119,0.6268292665481567,1.6868475675582886
66,0.73908615,0.9889080177528403,0.6353658437728882,1.5398529767990112
67,0.7262314,1.0045132035818873,0.6158536672592163,1.7074754238128662
68,0.7413771,0.9767554578730149,0.6219512224197388,1.483349323272705
69,0.74277717,0.9680130129038294,0.6219512224197388,1.9532339572906494
70,0.73730433,0.9730315151993852,0.6158536672592163,1.329857587814331
71,0.739468,0.976895222245729,0.6292682886123657,2.4270589351654053
72,0.73908615,0.966534267358536,0.6280487775802612,1.7197163105010986
73,0.735777,0.9789276592479917,0.6158536672592163,1.4507873058319092
74,0.7420135,0.9553382814317407,0.6304877996444702,1.6497560739517212
75,0.74214077,0.9447179906344283,0.6439024209976196,1.5781136751174927
76,0.7500318,0.9355357916995104,0.6439024209976196,1.3356094360351562
77,0.7429044,0.952266313684674,0.6121951341629028,1.2956383228302002
78,0.7476136,0.9379745349790571,0.6292682886123657,1.998668909072876
79,0.7460863,0.9331161820874864,0.6121951341629028,1.6319738626480103
80,0.75690466,0.9060455191919818,0.6365853548049927,1.8054636716842651
81,0.75092274,0.927574249560769,0.6256097555160522,1.7896440029144287
82,0.7528319,0.9132236767865113,0.6195122003555298,1.5852742195129395
83,0.7541046,0.9110783722711092,0.6487804651260376,1.802807092666626
84,0.7576683,0.8902263466723475,0.6195122003555298,1.6518384218215942
85,0.7546137,0.9007904997942511,0.6329268217086792,1.6659202575683594
86,0.761232,0.8820948022398403,0.6048780679702759,1.6598460674285889
87,0.75448644,0.8960568942200733,0.6353658437728882,1.8333407640457153
88,0.7695049,0.8499741472708779,0.6231707334518433,1.576465368270874
89,0.75754106,0.8710296044174278,0.6024390459060669,2.1270060539245605
90,0.759832,0.8787873889466226,0.6182926893234253,2.5192947387695312
91,0.7641593,0.8604023747731214,0.6487804651260376,1.7899302244186401
92,0.76925033,0.8540378613706853,0.6158536672592163,1.3688424825668335
93,0.76225024,0.8770227311630657,0.6341463327407837,1.3711484670639038
94,0.7697595,0.8523452214055561,0.6292682886123657,2.3935837745666504
95,0.7619957,0.8752951717443468,0.6268292665481567,1.711483120918274
96,0.7673412,0.8547873261230691,0.6268292665481567,1.7608572244644165
97,0.76785034,0.8523025380847441,0.6390243768692017,1.8692091703414917
98,0.770014,0.8439303914356948,0.6329268217086792,1.3853440284729004
99,0.7768869,0.8124584392371244,0.6292682886123657,2.273219347000122
100,0.773705,0.8395925430573624,0.6353658437728882,2.3406190872192383
101,0.76428664,0.8605588414339189,0.6365853548049927,1.6800363063812256
102,0.7709049,0.8379254640855603,0.6268292665481567,1.2173869609832764
103,0.7744686,0.832678386246231,0.6207317113876343,1.9066429138183594
104,0.77281404,0.8303879995799419,0.6426829099655151,2.258819818496704
105,0.77166855,0.8170269621424476,0.6475609540939331,1.3645354509353638
106,0.7735777,0.8345869494801459,0.6487804651260376,2.0284199714660645
107,0.7790505,0.8063037597800813,0.6439024209976196,1.7383135557174683
108,0.785287,0.7930195679855176,0.6426829099655151,1.8529839515686035
109,0.77994144,0.7997486195340325,0.6280487775802612,1.4981567859649658
110,0.77637774,0.817387262760144,0.6365853548049927,1.8011090755462646
111,0.7796869,0.8183879762019337,0.6256097555160522,2.1881778240203857
112,0.77637774,0.8035623637246497,0.6292682886123657,2.109757900238037
113,0.7780323,0.8037101907843592,0.6256097555160522,1.8236486911773682
114,0.78095967,0.7949128788027643,0.6219512224197388,1.848154902458191
115,0.7798142,0.7928122901819401,0.6304877996444702,1.7255101203918457
116,0.77319586,0.7922676529944421,0.6317073106765747,2.675750732421875
117,0.78388697,0.784369871120834,0.6548780202865601,2.0618627071380615
118,0.79012346,0.7867157237590747,0.6304877996444702,1.9432086944580078
119,0.78414154,0.7828377769757154,0.6280487775802612,2.2081422805786133
120,0.7884689,0.7633522822912434,0.6195122003555298,2.010404109954834
121,0.78350514,0.7773677566298186,0.6317073106765747,2.3863472938537598
122,0.7833779,0.7828655260975039,0.6317073106765747,1.8766523599624634
123,0.7893598,0.7680070625547691,0.6353658437728882,2.0507001876831055
124,0.7863052,0.7738482202223668,0.6329268217086792,1.0313364267349243
125,0.78414154,0.7651512772508655,0.6182926893234253,2.1111316680908203
126,0.78350514,0.7820024193433218,0.6195122003555298,1.9576092958450317
127,0.78299606,0.7645065890205746,0.6268292665481567,2.0589540004730225
128,0.7944508,0.7453487917441395,0.6121951341629028,1.279485821723938
129,0.791778,0.7513198321582071,0.6292682886123657,1.6368467807769775
130,0.7905053,0.7651316290049209,0.6243902444839478,1.3458640575408936
131,0.7973781,0.7344554129699206,0.6109756231307983,1.9727288484573364
132,0.7916508,0.7588364461063959,0.6512194871902466,1.556300401687622
133,0.7958508,0.7316144483453887,0.6243902444839478,2.1749720573425293
134,0.791778,0.7391921008332656,0.6280487775802612,2.0161359310150146
135,0.79865086,0.7397304516234925,0.6231707334518433,1.3550560474395752
136,0.80043274,0.7203163849999465,0.6219512224197388,1.4251306056976318
137,0.7935599,0.7355746732226034,0.6329268217086792,1.0842924118041992
138,0.7924144,0.7290403048273291,0.6280487775802612,1.4615795612335205
139,0.7973781,0.721668765595567,0.6304877996444702,1.5927702188491821
140,0.7875779,0.7504895827735943,0.6292682886123657,2.0338475704193115
141,0.79992366,0.7201832417041666,0.6109756231307983,2.609408140182495
142,0.791778,0.7397790446875583,0.6256097555160522,2.852426767349243
143,0.7971236,0.7243994674989706,0.6353658437728882,1.781339406967163
144,0.7945781,0.7284995216866311,0.6439024209976196,1.6737289428710938
145,0.8009418,0.715324054635844,0.6463414430618286,2.036245822906494
146,0.80666924,0.689120236055472,0.6353658437728882,2.0460474491119385
147,0.79623264,0.720590460136584,0.6097561120986938,2.4730427265167236
148,0.80692375,0.7001350662537441,0.6341463327407837,2.4475390911102295
149,0.79865086,0.7116093368753004,0.6134146451950073,1.8200085163116455
150,0.8001782,0.7148574047967406,0.6390243768692017,2.031444787979126
151,0.79508716,0.7338473060301914,0.6414633989334106,1.9968488216400146
152,0.80246913,0.7130482937978002,0.6243902444839478,2.158484935760498
153,0.79686904,0.7147610709687536,0.6390243768692017,1.925788164138794
154,0.7976327,0.7024884736232969,0.6414633989334106,3.272883415222168
155,0.80845106,0.6932149220623557,0.6256097555160522,1.5255707502365112
156,0.8011964,0.7057749771716805,0.6329268217086792,1.8007169961929321
157,0.80043274,0.6983128789728404,0.6329268217086792,1.8912965059280396
158,0.8003055,0.7043165707460748,0.6451219320297241,1.9812273979187012
159,0.7994145,0.7053689031909255,0.6292682886123657,2.595395565032959
160,0.8009418,0.7052430556720793,0.6439024209976196,2.491027593612671
161,0.8025964,0.6928004290032845,0.6621951460838318,1.4996798038482666
162,0.80246913,0.6993047817071704,0.6451219320297241,2.9633774757385254
163,0.80997837,0.6972242265131784,0.6378048658370972,1.7069604396820068
164,0.810742,0.6737694862888137,0.6256097555160522,1.7259547710418701
165,0.8117602,0.6764395436864933,0.6048780679702759,2.8513035774230957
166,0.804251,0.7016467057340121,0.6536585092544556,1.834092617034912
167,0.80756015,0.6689290142217119,0.6292682886123657,1.9781944751739502
168,0.8003055,0.6953948510577694,0.6682927012443542,1.4240546226501465
169,0.8108693,0.6665959246698433,0.6487804651260376,1.8299719095230103
170,0.8164694,0.6603688742651107,0.6085366010665894,1.877697467803955
171,0.8098511,0.6846932935793618,0.6195122003555298,2.112840175628662
172,0.81201476,0.6556761881937063,0.6426829099655151,1.5914521217346191
173,0.8143057,0.6675122627793825,0.6487804651260376,1.9211223125457764
174,0.8045055,0.6865733923067858,0.6585366129875183,1.260071039199829
175,0.8020873,0.6977128373297538,0.6426829099655151,2.2351622581481934
176,0.8067965,0.6962310619840978,0.6268292665481567,1.2258070707321167
177,0.8092147,0.6611513415828254,0.6463414430618286,1.607971429824829
178,0.8123966,0.6582394100880219,0.6463414430618286,1.7366368770599365
179,0.8150694,0.656693531191044,0.6243902444839478,1.6460260152816772
180,0.81226933,0.6578000098963768,0.6353658437728882,1.8544737100601196
181,0.8183785,0.6500888147532553,0.6414633989334106,1.6735684871673584
182,0.8157057,0.6576673326957191,0.6378048658370972,1.6420252323150635
183,0.8207967,0.6425836704395436,0.6426829099655151,1.9356964826583862
184,0.8163421,0.6590205937362255,0.6378048658370972,2.3974592685699463
185,0.81290567,0.6550127958104373,0.6548780202865601,2.582035541534424
186,0.8092147,0.6724566946628061,0.6609756350517273,2.2640557289123535
187,0.8150694,0.6525843532937803,0.6390243768692017,1.594382643699646
188,0.81379664,0.6691555129642688,0.6378048658370972,0.9985026121139526
189,0.8087056,0.6524281084074749,0.6256097555160522,1.692610263824463
190,0.81774217,0.6497136834803559,0.6304877996444702,1.8724040985107422
191,0.8192694,0.6450671797886868,0.6524389982223511,1.4737532138824463
192,0.80997837,0.6586021508320272,0.6658536791801453,1.5507947206497192
193,0.8148148,0.6526583068028227,0.6317073106765747,2.367157459259033
194,0.8261423,0.6285652921402198,0.6158536672592163,1.885461688041687
195,0.81736034,0.6491499471955738,0.6280487775802612,2.4732866287231445
196,0.82028764,0.6369807454516175,0.6353658437728882,1.4597327709197998
197,0.81825125,0.6387523887675425,0.6243902444839478,1.407548427581787
198,0.8230877,0.6119342139048841,0.6304877996444702,2.843863010406494
199,0.8204149,0.612516485940747,0.6548780202865601,2.2165164947509766

In [ ]:
## backup
# from google.colab import drive
# drive.mount('/content/gdrive')
# import os
# os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
# %tensorflow_version 2.x
# !pip install tensorflow --upgrade
# !pip install keras --upgrade
# !wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64 -O cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
# !apt-key add /var/cuda-repo-10-0-local-10.0.130-410.48/7fa2af80.pub
# !apt-get update
# !apt-get install cuda
# !pip install tf-nightly-gpu-2.0-preview
# !pip install tf-nightly-2.0-preview
# !unzip 101_ObjectCategories_resized.zip 
# !rm -rf __MACOSX
# !find -type f -name *.jpg | wc -l

## dataloader
# from sklearn.model_selection import train_test_split
# images = []
# # images_path = []
# labels = []
# img_root="101_ObjectCategories_resized/"
# label_names = next(os.walk(img_root))[1]
# n_classes = len(label_names) # 101
# for i in range(len(label_names)):
#         img_list = glob.glob(os.path.join(img_root, label_names[i], "*"))
#         for img_path in img_list:
#                 img = np.asarray(Image.open(img_path), dtype=np.float32) / 255.0
#                 images.append(img)
#                 # images_path.append(img_path)
#                 temp = np.zeros(n_classes, dtype=int)
#                 temp[i] = 1
#                 labels.append(temp)
# print(labels[1000])
# print(images_path[1000])
# print(label_names[np.argmax(labels[1000])])
# X = np.array(images) # (8677, 64, 64, 3)
# y = np.array(labels) # (8677, 101)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=0)
# print("Image Shape:{}".format(X.shape))
# print("Label Shape:{}".format(y.shape))

## fit model
# history = model.fit(X, y,
#         batch_size = 512,
#         epochs = 50,
#         callbacks=[checkpointer],
#         validation_split = 0.1)

## predict image
# test_img_path = 'test_set/anchor.jpg'
# test_img = image.load_img(test_img_path, target_size=(64, 64))
# test_img_tensor = image.img_to_array(test_img)
# test_img_tensor = np.expand_dims(test_img_tensor, axis=0)
# y_predict = model.predict(test_img_tensor)
# print("Predicted class is:",label_names[np.argmax(y_predict)])